In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('data.csv', index_col=0)

In [15]:
#df.sort_values('year', ascending=True)
ser = df['year'].value_counts().sort_index(ascending=False)
check = 0
years = []
for items in ser.iteritems(): 
    if check <= 50000:
        years.append(items[0])
        check += items[1]

In [17]:
years

[2020,
 2019,
 2018,
 2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2002,
 2001,
 2000,
 1999,
 1998,
 1997,
 1996,
 1995]

In [18]:
check

51470